Importing Libraries


In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

Creating Neural Network Architecture 

In [ ]:
class NeuralNetwork(nn.Module):
    
    def __init__( self, state_size, action_size, seed = 42 ) -> None:
        super( NeuralNetwork, self ).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)
        
    def forward(self, state):
        x = self.fc1(state)
        x = F.relu(x)
        
        #Takes input from first connected layer 
        x = self.fc2(x)
        x = F.relu(x)
        
        return self.fc3(x)

Training The Ai


In [ ]:
#Importing Libraries
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n

#insuring perameter values
print('State Shape: ', state_shape)
print('State Size: ', state_size)
print('Number of Actions: ', number_actions)

In [ ]:
#Initializing Hyperparamters

learning_rate = 5e-4
minibatch_size = 100 
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

In [ ]:
#Implementing Experience Replay

class ReplayMemory(object):
    
    def __init__(self, capacity) -> None:
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []
        
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
            
    def sample(self, batch_size):
        experiences = random.sample(self.memory, k = batch_size)
        #Stacking states together
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        
        return states, next_states, actions, rewards, dones
    

In [ ]:
#Implementing DQN class

class Agent():
    
    def __init__(self, state_size, action_size) -> None:
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        
        #Q-learning
        self.local_qNetwork = NeuralNetwork(state_size, action_size).to(self.device)
        self.target_qNetwork = NeuralNetwork(state_size, action_size).to(self.device)
        
        #Optimizer
        self.optimizer = optim.Adam(self.local_qNetwork.parameters(), lr = learning_rate)
        
        #Memory of Ai
        self.memory = ReplayMemory(replay_buffer_size)
        
        self.time_step = 0
        
    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.time_step = (self.time_step + 1) % 4
        if self.time_step == 0:
            if len(self.memory.memory) > minibatch_size:
                experiences = self.memory.sample(100)
                self.learn(experiences, discount_factor)
                
    def act(self, state, epsilon = 0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_qNetwork.eval()
        with torch.no_grad():
            action_values = self.local_qNetwork(state)
        self.local_qNetwork.train()
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
    def learn(self, experiences, discount_factor):
        states, next_states, actions, rewards, dones = experiences
        next_q_targets = self.target_qNetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_target = rewards + (discount_factor * next_q_targets * (1 - dones))
        q_expected = self.local_qNetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_target)
        self.optimizer.zero_grad()
        
        #Backpropigating
        loss.backward()
        
        self.optimizer.step()
        self.soft_update(self.local_qNetwork, self.target_qNetwork, interpolation_parameter)
        
    def soft_update(self, local_model, target_model, interpolation_parameter):
        for target_parameter, local_parameter in zip(target_model.parameters(),local_model.parameters()):
            target_parameter.data.copy_(interpolation_parameter * local_parameter.data + (1.0 - interpolation_parameter) * target_parameter.data)

In [ ]:
#Initializing DQN agent
agent = Agent(state_size, number_actions)

In [8]:
#Training DQN agent
number_episodes = 2000
maximum_timesteps_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
    state, _ = env.reset()
    score = 0 
    for t in range(maximum_timesteps_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
    
    print('\rEpisoode: {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
    if episode % 100 == 0:
        print('\rEpisoode: {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    if np.mean(scores_on_100_episodes) >= 200.0:
        print('\nEnviornment solved in  {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
        torch.save(agent.local_qNetwork.state_dict(), 'checkpoint.pth')
        break

Episoode: 100	Average Score: -160.31
Episoode: 200	Average Score: -107.16
Episoode: 300	Average Score: -26.637
Episoode: 400	Average Score: 48.962
Episoode: 500	Average Score: 97.29
Episoode: 600	Average Score: 169.72
Episoode: 700	Average Score: 192.18
Episoode: 739	Average Score: 200.40
Enviornment solved in  639 episodes!	Average Score: 200.40


Visualizing Results

In [16]:
import glob
import os
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

os.environ['IMAGEIO_FFMPEG_EXE'] = '/opt/homebrew/bin/ffmpeg'

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
